REsizing

In [2]:
from CellClicker.cell_clicker import ImageViewer
import tkinter as tk
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageViewer(root)
    root.mainloop()


XML file 'E:\Scott\Data\20240417\images\cell_reigons.xml' already exists.
XML file 'E:\Scott\Data\20240417\images\cell_reigons.xml' already exists.
